In [ ]:
import json
import string
import random
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer
import tensorflow as tensorF
from tensorflow import keras
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
ourData = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": [
                "Hello",
                "Hi",
                "Hey",
                "Hhat's up?",
                "Good morning",
                "Good evening",
                "How are you?",
                "How's it going?"
            ],
            "responses": [
                "Hello",
                "Hey!",
                "Hii",
                "Hello, human.",
                "How may I help you?"
            ]
        },
        {
            "tag": "goodbye",
            "patterns": [
                "bye",
                "Bye",
                "see you",
                "goodbye"
            ],
            "responses": [
                "See you later, thanks for visiting.",
                "Have a nice day!!",
                "Bye, see you soon."
            ]
        },
        {
            "tag": "thanks",
            "patterns": [
                "Thanks",
                "Thank you",
                "That's helpful",
                "Thank's a lot!"
            ],
            "responses": [
                "Happy to help!",
                "Any time!",
                "My pleasure"
            ]
        },
        {
            "tag": "items",
            "patterns": [
              "Which items do you have?",
              "What kinds of items are there?",
              "What do you sell?",
              "What's on menu",
              "Menu"
            ],
            "responses": [
              "Lots of delicious stuffs, clean, safe and out of love!!\nMaggi\tPasta\nMomos\tSpring Rolls\nBurger\tPaneer Kulcha\nPatties\tSandwich"
            ]
          },
          {
            "tag": "delivery",
            "patterns": [
              "How long does delivery take?",
              "When do I get my delivery?"
            ],
            "responses": [
              "A few 10 minutes",
              "A few minutes of wait ~ nearly 10 :)"
            ]
          }
    ]
}


In [ ]:
lm = WordNetLemmatizer() #for getting words
# lists
ourClasses = []
newWords = []
documentX = []
documentY = []
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in ourData["intents"]:
    for pattern in intent["patterns"]:
        ourTokens = nltk.word_tokenize(pattern)
        newWords.extend(ourTokens)
        documentX.append(pattern)
        documentY.append(intent["tag"])


    if intent["tag"] not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
ourClasses = sorted(set(ourClasses))# sorting classes

In [ ]:

print(newWords)

["'s", 'a', 'are', 'bye', 'delivery', 'do', 'doe', 'evening', 'get', 'going', 'good', 'goodbye', 'have', 'hello', 'helpful', 'hey', 'hhat', 'hi', 'how', 'i', 'it', 'item', 'kind', 'long', 'lot', 'menu', 'morning', 'my', 'of', 'on', 'see', 'sell', 'take', 'thank', 'thanks', 'that', 'there', 'up', 'what', 'when', 'which', 'you']


In [ ]:
print(ourClasses)

['delivery', 'goodbye', 'greeting', 'items', 'thanks']


In [ ]:
trainingData = [] # training list array
outEmpty = [0] * len(ourClasses)
# BOW model
for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)# coverting our data into an array afterv shuffling

x = num.array(list(trainingData[:, 0]))# first trainig phase
y = num.array(list(trainingData[:, 1]))# second training phase

In [ ]:
# defining some parameters
iShape = (len(x[0]),)
oShape = len(y[0])

# the deep learning model
ourNewModel = keras.models.Sequential()
ourNewModel.add(keras.layers.Dense(128, input_shape=iShape, activation="relu"))
ourNewModel.add(keras.layers.Dropout(0.5))
ourNewModel.add(keras.layers.Dense(64, activation="relu"))
ourNewModel.add(keras.layers.Dropout(0.3))
ourNewModel.add(keras.layers.Dense(oShape, activation = "softmax"))
md = keras.optimizers.Adam(learning_rate=0.01)
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
print(ourNewModel.summary())
ourNewModel.fit(x, y, epochs=200, verbose=1)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 128)               5504      
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 5)                 325       
                                                                 
Total params: 14,085
Trainable params: 14,085
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
1/1 [============================

In [ ]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def pred_class(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["intents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      ourResult = random.choice(i["responses"])
      break
  return ourResult

In [ ]:
while True:
    newMessage = input("")
    intents = pred_class(newMessage, newWords, ourClasses)
    ourResult = getRes(intents, ourData)
    print(ourResult)
    if(newMessage=='exit'):
      break


hello
1/1 [==============================] - 0s 59ms/step
Hey!
what items you have
1/1 [==============================] - 0s 22ms/step
Lots of delicious stuffs, clean, safe and out of love!!
Maggi	Pasta
Momos	Spring Rolls
Burger	Paneer Kulcha
Patties	Sandwich
i want maggi 
1/1 [==============================] - 0s 20ms/step
Hii
delivery
1/1 [==============================] - 0s 23ms/step
A few minutes of wait ~ nearly 10 :)
thanks
1/1 [==============================] - 0s 21ms/step
My pleasure
good bye
1/1 [==============================] - 0s 20ms/step
Have a nice day!!
exit
1/1 [==============================] - 0s 22ms/step
Hii
